# 5章 最適化問題の全体像を捉えよう
この章では、シフトスケジューリング問題を中心にした様々な最適化問題を解く方法を学んでいきます。

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter5

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements.txt

## 5-2 ソルバーを利用して線形最適化問題を解いてみよう

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals
from IPython.display import display

# データ読み込み
df_n = pd.read_csv('nutrition.csv', index_col="食料品")
df_p = pd.read_csv('price.csv')
print("食料品と栄養素の関係")
display(df_n)
print("食料品の価格")
display(df_p)

# 初期設定 #
np.random.seed(1)
np = len(df_n.index)
nn = len(df_n.columns)
pr = list(range(np))

# 数理モデル作成 #
m1 = model_min()
# 目的関数
v1 = {(i):LpVariable('v%d'%(i),cat='Integer',lowBound=0) for i in pr}
# 制約条件
m1 += lpSum(df_p.iloc[0][i]*v1[i] for i in pr)
for j in range(nn):
    m1 += lpSum(v1[i]*df_n.iloc[i][j] for i in range(np)) >= 100
m1.solve()

# 総コスト計算 #
print("最適解")
total_cost = 0
for k,x in v1.items():
    i = k
    print(df_n.index[i],"の個数:",int(value(x)),"個")
    total_cost += df_p.iloc[0][i]*value(x)

print("総コスト:",int(total_cost),"円")

## 5-3. 非線形最適化問題を解いてみよう 

### 二分探索により1000の二乗根を求める

In [ ]:
def f(x):
    return x**2 - 1000 

# 初期設定
lo = -0.1
hi = 1000.1
eps = 1e-10 # 許容誤差

# 二分探索を実行
count = 0
while hi-lo > eps:
    x = (lo + hi) / 2
    if f(x) >= 0:
        hi = x 
    else:
        lo = x 
    count += 1

print(f'結果: {hi}')
print(f'探索回数: {count}回')

### ニュートン法により1000の二乗根を求める

In [ ]:
# ニュートン法の関数
# x0, epsはデフォルト値
def square_root(y, x0=1, eps=1e-10):
    x = x0
    count = 0
    while abs(x**2 - y) > eps:
        x -= (x*x - y) / (2*x)
        count += 1
    return x, count

# ニュートン法の実行
x, count = square_root(1000)
print(f'結果: {x}')
print(f'探索回数: {count}回')

## 5-4. 従業員のシフトを自動で決める方法について検討してみよう 

### シフト希望表の読み込み

In [ ]:
import pandas as pd 
def schedules_from_csv(path):
    return pd.read_csv(path, index_col=0)

schedules_from_csv('schedule.csv')

## 5-5. シフト希望をグラフネットワークで可視化してみよう 

In [ ]:
import numpy as np

print(f'1 ~ 100までの整数を100個乱数生成')
a = np.random.randint(1, 100, 100)
print(a)

l = list(a)
s = set(a)

x = 50

# 下の2行はどちらも同じ結果になる
print(f'50が含まれているか: {x in l}')
print(f'50が含まれているか: {x in s}')

In [ ]:
%%time
for _ in range(10**6):
    x in l # listの場合 

In [ ]:
%%time 
for _ in range(10**6):
    x in s # setの場合

## 5-9 最大流問題を解いてみよう 

### 最大流を求めるコード

In [ ]:
from collections import deque
inf = float('inf')

class Graph:
    class __Edge:
        def __init__(self, capacity=1, **args):
            self.capacity = capacity

            
    def __init__(self, n=0):
        self.__N = n
        self.edges = [{} for _ in range(n)]

    
    # 辺を追加する関数
    def add_edge(self, u, v, **args): 
        self.edges[u][v] = self.__Edge(**args)

    
    # BFS(幅優先探索)を行う関数
    def bfs(self, src=0):
        n = self.__N
        self.lv = lv = [None]*n
        lv[src] = 0
        q = deque([src]) # BFSではqueueというデータ構造を使用する(Pythonではdequeue)
        while q:
            u = q.popleft()
            for v, e in self.edges[u].items():
                if e.capacity == 0: continue # フローを流すことができない(辺が存在しない)
                if lv[v] is not None: continue # すでにレベルが確定している
                lv[v] = lv[u] + 1
                q.append(v)
    
    # DFS(深さ優先探索)を行う関数
    def flow_to_sink(self, u, flow_in, sink):
        if u == sink:
            return flow_in
        flow = 0
        for v, e in self.edges[u].items():
            if e.capacity == 0: continue
            if self.lv[v] <= self.lv[u]: continue 
            f = self.flow_to_sink(v, min(flow_in, e.capacity), sink)
            if not f: continue
            self.edges[u][v].capacity -= f
            if u in self.edges[v]:
                self.edges[v][u].capacity += f
            else:
                self.add_edge(v, u, capacity=f)
            flow_in -= f
            flow += f
        return flow

    
    # 最大流が求まるまでBFSとDFSを繰り返し実行する
    def dinic(self, src, sink, visualize=False):
        flow = 0
        while True:
            if visualize:
                self.visualizer(self)
            self.bfs(src)
            if self.lv[sink] is None:
                return flow
            flow += self.flow_to_sink(src, inf, sink)
            
    # 可視化するための関数をセット
    def set_visualizer(self, visualizer):
        self.visualizer = visualizer

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt 
plt.figure(figsize=(10,5))

# 辺の設定
edges = [
    ((0, 2), 3),
    ((0, 1), 9),
    ((1, 2), 8),
    ((2, 3), 7),
    ((1, 4), 2),
    ((2, 5), 5),
    ((3, 4), 6),
    ((4, 5), 4),
    ((4, 6), 3),
    ((5, 6), 10)
]

# 頂点座標を設定
nodes = [
    (0, 1),
    (1, 0),
    (1, 2),
    (2, 1),
    (3, 0),
    (3, 2),
    (4, 1),
]

n = len(nodes)

# 可視化用のグラフ
graph = nx.DiGraph()

# グラフに頂点番号を追加
graph.add_nodes_from(range(n))

# 頂点座標の情報をグラフに追加しやすい形に整形
pos = dict(enumerate(nodes))

# 最初の状態を描画
plt.figure(figsize=(10,5))

for (u, v), cap in edges:
    graph.add_edge(u, v, capacity=cap)

labels = nx.get_edge_attributes(graph,'capacity')
nx.draw_networkx_edge_labels(graph,pos,edge_labels=labels, font_color='r', font_size=20)
nx.draw_networkx(graph, pos=pos, node_color='c')
plt.show()
graph.remove_edges_from([e[0] for e in edges])

In [ ]:
# 最大流を解くためグラフを生成
g = Graph(n)

for (u, v), cap in edges: 
    g.add_edge(u, v, capacity=cap) # 辺を追加。

    
# 途中結果のフローの様子を描画する関数
def show_progress(g):
    plt.figure(figsize=(10,5))

    for (u, v), cap in edges:
        e = g.edges[u][v]
        if e.capacity >= cap:
            continue 
        graph.add_edge(u, v, capacity=cap-e.capacity)

    labels = nx.get_edge_attributes(graph,'capacity')
    nx.draw_networkx_edge_labels(graph,pos,edge_labels=labels, font_color='g', font_size=20)
    nx.draw_networkx(graph, pos=pos, node_color='c')
    plt.show()
    graph.remove_edges_from([e[0] for e in edges])

# 可視化するための関数をセット
g.set_visualizer(show_progress)

print(f'最大流: {g.dinic(src=0, sink=6, visualize=True)}')

## 5-10. 最大流問題を応用して、シフトスケジューリング問題を解いてみよう

### シフトの読み込み

In [ ]:
import numpy as np 

import pandas as pd 
def schedules_from_csv(path):
    return pd.read_csv(path, index_col=0)

schedules = schedules_from_csv('schedule.csv')
n, m = schedules.shape
schedules

### ネットワークの可視化

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt 

plt.figure(figsize=(20, 10))

# 可視化用のグラフ
graph = nx.DiGraph()

N = n + m + 2 
graph.add_nodes_from(range(N))
# n個の頂点をグラフに追加
center = 10
vertices = [(center,9)] + [(center + (i-n//2), 6) for i in range(n)] + [(center+ (i-m//2), 3) for i in range(m)] + [(center, 0)]


# 辺の作成
schedules = schedules.values
edges = np.argwhere(schedules)
edges += 1
edges[:,1] += n
edges1 = np.array([(0, i+1) for i in range(n)]).reshape(-1, 2)
edges2 = np.array([(i+n+1, n+m+1) for i in range(m)]).reshape(-1, 2)
edges = np.vstack([edges1, edges, edges2])

# 頂点座標の情報をグラフに追加しやすい形に整形
pos = dict(enumerate(vertices))

# 辺の追加
for u, v in edges:
    graph.add_edge(u, v, capacity=1)

# 描画
nx.draw_networkx(graph, pos=pos, node_color='c') 
plt.show()

### 最大流を求める

In [ ]:
g = Graph(N)

# 辺を追加
for u, v in edges: 
    g.add_edge(u, v, capacity=1)

print(f'最大流: {g.dinic(src=0, sink=N-1)}')

### 結果の可視化

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt 

plt.figure(figsize=(20, 10))

# 可視化用のグラフ
graph = nx.DiGraph()

N = n + m + 2 
graph.add_nodes_from(range(N))
center = 10

# 描画する座標を決める
vertices = [(center,9)] + [(center + (i-n//2), 6) for i in range(n)] + [(center+ (i-m//2), 3) for i in range(m)] + [(center, 0)]

# 辺の作成
edges = np.argwhere(schedules)
edges += 1
edges[:,1] += n

# 頂点座標の情報をグラフに追加しやすい形に整形
pos = dict(enumerate(vertices))

# シフト表の初期化
shift_table = np.zeros(shape=(n, m), dtype=np.int8)

# 辺を追加
for u, v in edges:
    e = g.edges[u][v]
    if e.capacity == 1:# マッチングしていない辺は描画しない
        continue
    graph.add_edge(u, v, capacity=1)
    u -= 1 # 従業員のindexに変換
    v -= 1 + n # シフトのindexに変換
    shift_table[u, v] = 1

# 描画
nx.draw_networkx(graph, pos=pos, node_color='c')
plt.show()

### 結果のシフト表を出力

In [ ]:
# データフレームに変換
shift_table = pd.DataFrame(shift_table)

# カラムとインデックスを設定
idx = [f'候補者{i}' for i in range(n)]
col = [
    f'{day}{time}'
    for day in ['月', '火', '水', '木', '金']
    for time in ['朝', '昼', '夜']
]
shift_table.rename(index=dict(enumerate(idx)), columns=dict(enumerate(col)))